# Script for Jetbot with Road Following und Collision Avoidance combined with TensorRT

### Libraries

Import all necessary libraries for this notebook in one central place

External libraries:
+ https://github.com/NVIDIA-AI-IOT/torch2trt/tree/v0.3.0
+ https://github.com/NVIDIA-AI-IOT/jetbot

In [ ]:
import math
import os
import time
import traitlets
from uuid import uuid1

import ipywidgets as widgets
from IPython.display import display

import PIL.Image

import torch
import torch.nn.functional as functional
import torchvision.transforms as transforms

from torch2trt import TRTModule

from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot

### Logic Part I (Pre-Requirements)

Load both TRT optimized models for Road Following and Collision Avoidance

In [ ]:
device = torch.device('cuda')

model_road = TRTModule()
model_road.load_state_dict(torch.load('models/road_following/best_steering_model_xy_17_04_a_trtv2.pth'))

model_collision = TRTModule()
model_collision.load_state_dict(torch.load('models/collision_resnet/collision_model_resnet_19_04_a_trt.pth'))

Init the Robot object

In [ ]:
robot = Robot()

Define some global variables

In [ ]:
angle_last, stop_counter = 0.0, 0
can_drive = True
stop_counter_limit = 10
x, y = 0.0, 0.0
robot_is_stopped = True

Create a Camera instance

In [ ]:
camera = Camera.instance(width=224, height=224, fps=10)

Preprocess a given image

In [ ]:
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Define a function for taking a snapshot

*Note: Can be used for taking an image when the Jetbot is blocked via Collision Avoidance but shouldn't,
so the taken snapshot can later be put in the "free" folder of the training dataset for the next Collision Avoidance training.*

In [ ]:
def take_snapshot(b=None):
    try:
        os.makedirs("snapshot")
    except FileExistsError:
        pass
    with open(os.path.join('snapshot', str(uuid1()) + '.jpg'), 'wb') as f:
        f.write(image_widget.value)

Calculate speed values

In [ ]:
def calculate_speed(last_a: float, x_in: float, y_in: float) -> (float, float, float):
    global a_slider

    # see start_stop_toggle()
    if robot_is_stopped:
        return 0.0, 0.0, 0.0

    # calculate alpha value
    a = math.atan2(x_in, y_in)
    a_slider.value = a
    xy_out.value = f"({x:.02f} {y:.02f})"

    # get the current PointOfInterest and and calculate the resulting steering parameters
    pid = a * steer_gain + (a - last_a) * steer_kd_gain
    steer_val = pid + steer_bias
    left = max(min(speed_control + steer_val, 1.0), 0.0)
    right = max(min(speed_control - steer_val, 1.0), 0.0)

    return a, left, right

Handle Collision Avoidance from single camera frame

In [ ]:
def get_collision_chance(image) -> float:
    collision = model_collision(image)
    collision_softmax = functional.softmax(collision, dim=1).flatten()

    return float(collision_softmax[0])

Handle Road Following from single camera frame

In [ ]:
def get_road_direction(image) -> (float, float):
    model_xy = model_road(image).detach().float().cpu().numpy().flatten()
    model_x = model_xy[0]
    model_y = (0.5 - model_xy[1]) / 2.0

    return model_x, model_y

### UI Elements

Create widgets for information and camera feed display

In [ ]:
image_widget = widgets.Image()

speed_output = widgets.Text(
    value='',
    placeholder='',
    description='(l,r):',
    disabled=True
)

model_output = widgets.Textarea(
    value='',
    placeholder='placeholder',
    description='Model:',
    disabled=False
)

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

Define sliders and information widgets

*Note: All default values that should lead to good driving results are defined as value=x*

In [ ]:
# Road Following sliders
speed_control_slider = widgets.FloatSlider(value=0.12, min=0.0, max=1.0, step=0.01, description='speed control')
steering_gain_slider = widgets.FloatSlider(value=0.03, min=0.0, max=1.0, step=0.01, description='steering gain')
steering_kd_gain_slider = widgets.FloatSlider(value=0.06, min=0.0, max=0.5, step=0.001, description='steering kd')
steering_bias_slider = widgets.FloatSlider(value=0.0, min=-0.3, max=0.3, step=0.01, description='steering bias')

# Collision Avoidance sliders
blocked_slider = widgets.FloatSlider(min=0.0, max=1.0, orientation='horizontal', description='blocked')
stop_duration_slider = widgets.IntSlider(value=10, min=1, max=1000, step=1, description='time for stop')
blocked_threshold = widgets.FloatSlider(value=0.8, min=0, max=1.0, step=0.01, description='blocked threshold')

# Steering sliders
diff_gain_limit_slider = widgets.FloatSlider(value=1.5, min=0.0, max=1.8, step=0.01, description='diff_gain_limit')
curve_min_val_slider = widgets.FloatSlider(value=0.08, min=0.0, max=0.25, step=0.01, description='curve_min_val')
alpha_factor_slider = widgets.FloatSlider(value=0.00, min=-1.0, max=1.0, step=0.01, description='alpha_factor')
steering_correction_activation = widgets.FloatSlider(value=0.25, min=0.0, max=math.pi, step=0.05, description='steering_correction_activation')

toggle_exec_btn = widgets.ToggleButton(
    value=False,
    description='Start',
    disabled=False,
    button_style='success',
    tooltip='Description',
    icon='check'
)

snapshot_btn = widgets.Button(
    description='snapshot'
)

Define the variables for slider controls

In [ ]:
speed_control, steer_gain, steer_kd_gain, steer_bias = speed_control_slider.value, 0.0, 0.0, 0.0

Create a debug slider for displaying the current speed (alpha value).

In [ ]:
a_slider = widgets.FloatSlider(min=-math.pi, max=math.pi, step=0.01, description='a')
xy_out = widgets.Text()

Attach functions to ui elements

### Logic Part II (Driving)

Execute the overall driving logic

In [ ]:
def execute(change):
    global angle_last, blocked_slider, robot, stop_counter, stop_counter_limit, can_drive, x, y, blocked_threshold
    global speed_control, steer_gain, steer_kd_gain, steer_bias

    # get image from current camera frame
    image = change['new']
    image_preproc = preprocess(image).to(device)

    # get slider values
    steer_gain = steering_gain_slider.value
    steer_kd_gain = steering_kd_gain_slider.value
    steer_bias = steering_bias_slider.value
    stop_counter_limit = stop_duration_slider.value
    max_speed = speed_control_slider.value

    prob_blocked = get_collision_chance(image_preproc)
    blocked_slider.value = prob_blocked

    # when the Jetbot can drive
    if can_drive:
        # Collision Avoidance factor determines if Jetbot can drive
        can_drive = prob_blocked <= blocked_threshold.value
        if can_drive:
            stop_counter = 0
            x, y = get_road_direction(image_preproc)
            speed_control = max_speed
        else:
            stop_counter += 1

    # when the Jetbot can not drive (e.g. is blocked by Collision Avoidance)
    else:
        stop_counter += 1
        if stop_counter < stop_counter_limit:
            x, y, speed_control = 0.0, 0.0, 0
        # move anyway after certain time (frames) has passed (stop_counter_limit)
        else:
            can_drive = True
            stop_counter = 0

    # calculate the steering parameters
    angle, left, right = calculate_speed(angle_last, x, y)
    angle_last = angle

    # move the robot
    robot.left_motor.value = left
    robot.right_motor.value = right

    # display the steering parameters
    speed_output.value = f'{left:.02f}/{right:.02f}'


execute({'new': camera.value})

Creates a toggle for starting and stopping the Jetbot

In [ ]:
def start_stop_toggle(obj):
    global robot_is_stopped
    if obj["new"]:
        toggle_exec_btn.button_style = 'warning'
        toggle_exec_btn.icon = 'warning'
        toggle_exec_btn.description = "Stop"
        camera.observe(execute, names='value')
        robot_is_stopped = False
    else:
        toggle_exec_btn.button_style = 'success'
        toggle_exec_btn.icon = 'check'
        toggle_exec_btn.description = 'Start'
        camera.unobserve(execute, names='value')
        robot_is_stopped = True
        time.sleep(0.1) # Not the finest solution, but this makes sure all model calculations are finished before robot is really stopped
        robot.stop()

### Display

In [ ]:
snapshot_btn.on_click(take_snapshot)

toggle_exec_btn.observe(start_stop_toggle, 'value')

Display all information

In [ ]:
display(speed_control_slider, steering_gain_slider, steering_kd_gain_slider, steering_bias_slider)

display(diff_gain_limit_slider, curve_min_val_slider, alpha_factor_slider, steering_correction_activation)

display(widgets.HBox([toggle_exec_btn, snapshot_btn]))

display(widgets.HBox([image_widget]))
display(speed_output)

display(widgets.HBox([blocked_slider, blocked_threshold, stop_duration_slider]))
display(a_slider, xy_out)